In [ ]:
%%sql

-- 0. CARGA DE DATOS (Sincronización de metadatos)

REFRESH TABLE lh_silver.fact_ventas;
REFRESH TABLE lh_silver.dim_direccion;


-- 1. MÉTRICAS GLOBALES (Capa Gold)

-- Creamos o reemplazamos la tabla de hechos en Gold
CREATE OR REPLACE TABLE lh_gold.fact_global_negocio
USING DELTA
AS
SELECT 
    producto_id, 
    fecha_id, 
    codigo_postal_id,
    proveedor_asignado,
    estado_logistica,
    SUM(monto_linea_total) AS ingresos_totales,
    SUM(cantidad) AS unidades_vendidas,
    SUM(gasto_logistico_total) AS gasto_logistico_total,
    AVG(precio_venta_real) AS precio_promedio_unidad,
    COUNT(orden_id) AS total_pedidos
FROM lh_silver.fact_ventas
GROUP BY 
    producto_id, 
    fecha_id, 
    codigo_postal_id,
    proveedor_asignado,
    estado_logistica;


-- 2. GUARDADO DE DIMENSIÓN EN GOLD

-- Trasladamos la dimensión de dirección a la capa Gold
CREATE OR REPLACE TABLE lh_gold.dim_direccion_gold
USING DELTA
AS
SELECT * FROM lh_silver.dim_direccion;

StatementMeta(, 44316b77-c34e-4a9f-99a2-d3e7328a226e, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>